In [5]:
!pip install dash pandas plotly

In [8]:
!pip install dash dash-core-components dash-html-components

In [1]:
!pip install requests

In [2]:
import requests

# Define the URLs
data_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
app_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"

# Define the local file names
data_file = "spacex_launch_dash.csv"
app_file = "spacex_dash_app.py"

# Download the data file
with open(data_file, "wb") as f:
    response = requests.get(data_url)
    f.write(response.content)

# Download the app file
with open(app_file, "wb") as f:
    response = requests.get(app_url)
    f.write(response.content)

print("Files downloaded!")

Files downloaded!


In [3]:
import pandas as pd

# Load the dataset into a pandas DataFrame
spacex_data = pd.read_csv("spacex_launch_dash.csv")

# Display the first few rows of the DataFrame
spacex_data.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [9]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd

# Assuming you have loaded your dataframe as 'spacex_df'
spacex_df = pd.read_csv("spacex_launch_dash.csv")

# Extract unique launch sites
launch_sites = spacex_df['Launch Site'].unique()

# Create options for dropdown
options_list = [{'label': 'All Sites', 'value': 'ALL'}]
for site in launch_sites:
    options_list.append({'label': site, 'value': site})

# Create the app layout
app = dash.Dash(__name__)
app.layout = html.Div([
    dcc.Dropdown(
        id='site-dropdown',
        options=options_list,
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    # Add additional components here if needed
])

if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\ilyas\AppData\Local\Temp\ipykernel_7416\3483862839.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\ilyas\AppData\Local\Temp\ipykernel_7416\3483862839.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [8]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Sample data
spacex_df = pd.read_csv("spacex_launch_dash.csv")

# Create a new Dash app instance
app = dash.Dash(__name__)

# Get unique launch sites
launch_sites = spacex_df['Launch Site'].unique().tolist()

# Define the layout of the app
app.layout = html.Div([
    dcc.Dropdown(
        id='site-dropdown',
        options=[{'label': 'All Sites', 'value': 'ALL'}] + [{'label': i, 'value': i} for i in launch_sites],
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    dcc.Graph(id='success-pie-chart')
])

# Define callback
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', names='Launch Site', title='Total Success Launches By Site')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(filtered_df, names='class', title=f'Total Success Launches for site {entered_site}')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [9]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Sample data
spacex_df = pd.read_csv("spacex_launch_dash.csv")

# Create a new Dash app instance
app = dash.Dash(__name__)

# Get unique launch sites
launch_sites = spacex_df['Launch Site'].unique().tolist()

# Define the layout of the app
app.layout = html.Div([
    dcc.Dropdown(
        id='site-dropdown',
        options=[{'label': 'All Sites', 'value': 'ALL'}] + [{'label': i, 'value': i} for i in launch_sites],
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    dcc.Graph(id='success-pie-chart'),
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        marks={
            0: '0',
            2500: '2500',
            5000: '5000',
            7500: '7500',
            10000: '10000'
        },
        value=[0, 10000]
    )
])

# Define callback
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def get_pie_chart(entered_site, payload_range):
    low, high = payload_range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)]
    
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='Total Success Launches By Site')
    else:
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig = px.pie(filtered_df, names='class', title=f'Total Success Launches for site {entered_site}')
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [10]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Sample data
spacex_df = pd.read_csv("spacex_launch_dash.csv")

# Create a new Dash app instance
app = dash.Dash(__name__)

# Get unique launch sites
launch_sites = spacex_df['Launch Site'].unique().tolist()

# Define the layout of the app
app.layout = html.Div([
    dcc.Dropdown(
        id='site-dropdown',
        options=[{'label': 'All Sites', 'value': 'ALL'}] + [{'label': i, 'value': i} for i in launch_sites],
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    dcc.Graph(id='success-pie-chart'),
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        marks={
            0: '0',
            2500: '2500',
            5000: '5000',
            7500: '7500',
            10000: '10000'
        },
        value=[0, 10000]
    ),
    dcc.Graph(id='success-payload-scatter-chart')
])

# Define callback for Pie chart
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def get_pie_chart(entered_site, payload_range):
    low, high = payload_range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)]
    
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='Total Success Launches By Site')
    else:
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig = px.pie(filtered_df, names='class', title=f'Total Success Launches for site {entered_site}')
    return fig

# Define callback for scatter plot
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def get_scatter_plot(entered_site, payload_range):
    low, high = payload_range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)]
    
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color="Booster Version Category", title='Payload vs. Outcome for All Sites')
    else:
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color="Booster Version Category", title=f'Payload vs. Outcome for {entered_site}')
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
